In [1]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import set_with_dataframe

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/spreadsheets",
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(credentials)
sheet1 = client.open("NBA Bets 2021-22").get_worksheet(1)

In [2]:
import pandas as pd
import pandasql as ps

In [3]:
mdf = pd.DataFrame(sheet1.get_all_records())

mdf

,participant full name,date,opponent,home,spread,moneyline,total,points scored,points allowed,total points scored,lost by,spread result,moneyline result,total result
0,L.A. Lakers,2021-10-03,Brooklyn Nets,TRUE,-4.0,-170,221.0,97,123,220,26,loss,loss,loss
1,Brooklyn Nets,2021-10-03,L.A. Lakers,FALSE,4.0,145,221.0,123,97,220,-26,win,win,loss
2,Miami Heat,2021-10-04,Atlanta Hawks,TRUE,-3.5,-165,212.5,125,99,224,-26,win,win,win
3,Atlanta Hawks,2021-10-04,Miami Heat,FALSE,3.5,140,212.5,99,125,224,26,loss,loss,win
4,Orlando Magic,2021-10-04,Boston Celtics,FALSE,7.0,230,213.5,97,98,195,1,win,loss,loss
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Milwaukee Bucks,2021-10-13,Utah Jazz,FALSE,1.5,140,222.5,120,124,244,4,loss,loss,win
104,Detroit Pistons,2021-10-13,New York Knicks,FALSE,7.5,265,218.5,100,108,208,8,loss,loss,loss
105,New York Knicks,2021-10-13,Detroit Pistons,TRUE,-7.5,-330,218.5,108,100,208,-8,win,win,loss
106,Portland Trail Blazers,2021-10-13,Phoenix Suns,FALSE,3.5,140,219.5,74,119,193,45,loss,loss,loss


In [4]:
mdf.dtypes


participant full name     object
date                      object
opponent                  object
home                      object
spread                   float64
moneyline                  int64
total                    float64
points scored              int64
points allowed             int64
total points scored        int64
lost by                    int64
spread result             object
moneyline result          object
total result              object
dtype: object

In [5]:
# I want the lakers' last 3 games record against the spread as a favorite

relevant_df = mdf.loc[(mdf['participant full name'] == 'L.A. Lakers') & (mdf['spread'] < 0)][-3:]

len(relevant_df.loc[relevant_df['spread result'] == 'win']), len(relevant_df.loc[relevant_df['spread result'] == 'loss']), len(relevant_df.loc[relevant_df['spread result'] == 'push'])

(0, 3, 0)

In [6]:
# get a team's record over last "games" amount of games (defaulted to all games) for specified bet (default to moneyline)

def team_record(dataframe, team, bet='moneyline result', games=82):
    spec_team = dataframe.loc[dataframe['participant full name'] == team][games*(-1):]
    return len(spec_team.loc[spec_team[bet] == 'win']), len(spec_team.loc[spec_team[bet] == 'loss']), len(spec_team.loc[spec_team[bet] == 'push'])

team_record(dataframe=mdf, team='L.A. Clippers', bet='total result')

(2, 2, 0)

In [7]:
def vs_opp(dataframe, team, opponent, bet='moneyline result', games=82):
    team_record(dataframe, team, bet, games)


In [8]:
def record_return(func, bet='moneyline result'):
    def wrapper_record_return(*args, **kwargs):
        spec_team = func(*args, **kwargs)
        return len(spec_team.loc[spec_team[bet] == 'win']), len(spec_team.loc[spec_team[bet] == 'loss']), len(spec_team.loc[spec_team[bet] == 'push'])
    return wrapper_record_return

def r(dataframe, team, games=82):
    return dataframe.loc[dataframe['participant full name'] == team][games*(-1):]

r(mdf, 'Golden State Warriors')

rr = record_return(r, 'spread')(mdf, 'Golden State Warriors')

rr

(0, 0, 0)

In [9]:
def f1(n):
    n += '1'
    return n

def f2(n):
    n += '2'
    return n

func_ls = [f1, f2]
a = 'a'
for func in func_ls:
    a = func(a)

a

'a12'

In [24]:
def team_func(dataframe, team):
    return dataframe.loc[dataframe['participant full name'] == team]

def opp_func(dataframe, opponent):
    return dataframe.loc[dataframe['opponent'] == opponent]

def last_games_func(dataframe, n):
    return dataframe.iloc[(-1 * n):, :]

def home_func(dataframe, home_or_not):
    return dataframe.loc[dataframe['home'] == home_or_not]

def spread_func(dataframe, spread_filt_settings: dict):
    d ={
        'exact': dataframe.loc[(dataframe['spread'] == spread_filt_settings['points'])],
        'longer': dataframe.loc[dataframe['spread'] >= spread_filt_settings['points']],
        'shorter': dataframe.loc[dataframe['spread'] <= spread_filt_settings['points']]
    }
    return d[spread_filt_settings['kind']]

def favorite_func(dataframe, favorite):
    if favorite:
        return dataframe.loc[dataframe['spread'] < 0]
    else:
        return dataframe.loc[dataframe['spread'] > 0]


In [25]:
filters = {
    'team': team_func,
    'opponent': opp_func,
    'last n games': last_games_func,
    'home': home_func,
    'spread': spread_func,
    'favorite': favorite_func

}

In [34]:
filter_these = {
    'bet': 'spread result',
    'favorite': True
}


In [35]:
df = mdf

for key in filters:
    if key in filter_these:
        df = filters[key](df, filter_these[key])

print("Record is", len(df.loc[df[filter_these['bet']] == 'win']), "wins", len(df.loc[df[filter_these['bet']] == 'loss']), "losses", len(df.loc[df[filter_these['bet']] == 'push']), "pushes")

df

Record is 24 wins 29 losses 1 pushes


,participant full name,date,opponent,home,spread,moneyline,total,points scored,points allowed,total points scored,lost by,spread result,moneyline result,total result
0,L.A. Lakers,2021-10-03,Brooklyn Nets,TRUE,-4.0,-170,221.0,97,123,220,26,loss,loss,loss
2,Miami Heat,2021-10-04,Atlanta Hawks,TRUE,-3.5,-165,212.5,125,99,224,-26,win,win,win
5,Boston Celtics,2021-10-04,Orlando Magic,TRUE,-7.0,-280,213.5,98,97,195,-1,loss,win,loss
7,Charlotte Hornets,2021-10-04,Oklahoma City Thunder,FALSE,-3.0,-165,215.5,113,97,210,-16,win,win,loss
8,Minnesota Timberwolves,2021-10-04,New Orleans Pelicans,TRUE,-3.0,-160,220.5,117,114,231,-3,push,win,win
10,Utah Jazz,2021-10-04,San Antonio Spurs,FALSE,-3.5,-165,217.5,85,111,196,26,loss,loss,loss
12,Portland Trail Blazers,2021-10-04,Golden State Warriors,FALSE,-3.0,-160,219.5,107,121,228,14,loss,loss,win
14,Phoenix Suns,2021-10-04,Sacramento Kings,FALSE,-1.5,-120,217.5,106,117,223,11,loss,loss,win
16,Denver Nuggets,2021-10-04,L.A. Clippers,FALSE,-1.5,-120,219.5,102,103,205,1,loss,loss,loss
19,Philadelphia 76ers,2021-10-04,Toronto Raptors,FALSE,-1.5,-120,216.5,107,123,230,16,loss,loss,win
